### 会话式 RAG

在许多问答应用程序中，我们希望允许用户进行来回对话，这意味着应用程序需要对过去的问题和答案进行某种“记忆”，以及将这些问题和答案纳入当前思维的逻辑。

在本指南中，我们重点介绍如何添加用于合并历史消息的逻辑。有关聊天记录管理的更多详细信息，请参阅此处。

我们将介绍两种方法：

`链`，我们总是在其中执行检索步骤;

`代理`，在代理中，我们赋予 LLM 是否以及如何执行检索步骤（或多个步骤）的自由裁量权。

对于外部知识源，我们将使用 RAG 教程中 Lilian Weng 撰写的 LLM Powered Autonomous Agents 博客文章。


#### 设置

依赖

在本演练中，我们将使用 阿里大模型 嵌入和 Chroma 矢量存储，但此处显示的所有内容都适用于任何嵌入`VectorStore` 或 `Retriever`

我们将使用以下包：

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma bs4

Note: you may need to restart the kernel to use updated packages.


In [39]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass() # 还是用阿里的啊
os.environ["DASHSCOPE_API_KEY"] = getpass.getpass() # 向量模型用阿里

# import dotenv

# dotenv.load_dotenv()

让我们首先重温一下我们在 RAG 教程中由 Lilian Weng 撰写的 LLM Powered Autonomous Agents 博客读后感的文章 -> 中文

In [44]:

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="qwen-plus", base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")  # 这次，我们还是用阿里

In [41]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
# from langchain_openai import OpenAIEmbeddings # 不用openai的，用阿里的
from langchain_community.embeddings import DashScopeEmbeddings 
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [42]:
# 1. Load, chunk and index the contents of the blog to create a retriever.
loader = WebBaseLoader(
    web_paths=("https://tttturtle-russ.github.io/2024/03/04/%E5%9F%BA%E4%BA%8ELLM%E9%A9%B1%E5%8A%A8%E7%9A%84%E8%87%AA%E5%8A%A8%E4%BB%A3%E7%90%86-1-Planning/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=DashScopeEmbeddings())
retriever = vectorstore.as_retriever()


# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    " 你是问答任务的助手。"
    " 使用以下检索到的上下文来回答"
    " 问题。如果你不知道答案，就说你"
    " 不知道。最多使用三句话，并保留"
    " 回答简明扼要。"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [43]:
response = rag_chain.invoke({"input": "什么是任务分解？"})
response["answer"]

'任务分解是将一个复杂的任务拆分成一系列更小、更易于管理的步骤的过程。在规划器的上下文中，这可以通过思维链（Chain of Thought, CoT）和思维树（Tree of Thought）来实现。思维链是让模型逐步思考，将大任务拆解成小步骤，而思维树则进一步扩展这一思想，对每个步骤生成多个可能的想法，形成一个树状结构。这样的分解有助于代理更好地理解和执行复杂的任务。'

请注意，我们使用了内置的链构造函数 和 ，因此我们的解决方案的基本成分是：`create_stuff_documents_chain`  `create_retrieval_chain`

retriever;

prompt;

LLM.

这将简化合并聊天记录的过程。

### 添加聊天记录

我们构建的链直接使用输入查询来检索相关上下文。但在对话设置中，用户查询可能需要理解对话上下文。例如，考虑以下交换：

- 人类：“什么是任务分解？

- AI：“任务分解涉及将复杂的任务分解为更小、更简单的步骤，以使它们更易于代理或模型管理。

- 人类：“常见的方法是什么？

为了回答第二个问题，我们的系统需要理解“它”指的是“任务分解”。

我们需要更新有关现有应用的两件事：

提示：更新提示以支持历史消息作为输入。

情境化问题：添加一个子链，该子链接受最新的用户问题，并在聊天记录的上下文中对其进行改革。

这可以简单地看作是构建一个新的“历史感知”猎犬。而在此之前，我们有：   query -> retriever

现在我们将拥有： (query, conversation history) -> LLM -> rephrased query -> retriever

##### 将问题置于情境中

首先，我们需要定义一个子链，该子链接受历史消息和最新的用户问题，并在问题引用了历史信息中的任何信息时对其进行改革。

我们将使用一个提示，其中包含名称为“chat_history”的变量。这允许我们使用“chat_history”输入键将消息列表传递给提示符，并且这些消息将插入到系统消息之后和包含最新问题的人工消息之前。`MessagesPlaceholder`

请注意，我们利用`create_history_aware_retriever`的帮助程序函数来执行此步骤，该函数管理为空的情况，否则按顺序应用。

`chat_historyprompt | llm | StrOutputParser() | retriever`

`create_history_aware_retriever` 构造一个接受键和作为输入的链，并具有与检索器相同的输出架构。`inputchat_history`

In [10]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    " 给定聊天记录和最新的用户问题"
    " 这可能引用了聊天记录中的上下文，"
    " 提出一个可以理解的独立问题 "
    " 没有聊天记录。不要回答问题. "
    " 如果需要，只需重新配制它，否则就按原样返回。"
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

此链将输入查询的改写预写到我们的检索器，以便检索包含对话的上下文。

现在，我们可以构建完整的 QA 链。这就像将猎犬更新为我们的新猎犬一样简单。`history_aware_retriever`

同样，我们将使用 `create_stuff_documents_chain` 生成一个带有输入键的 ，

并且 -- 它接受检索到的上下文以及对话历史记录和查询以生成答案。`question_answer_chain` `context`  `chat_historyinput`


In [11]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

让我们试试这个。下面我们提出一个问题和一个后续问题，需要情境化才能返回合理的回答。

由于我们的链包含输入，因此呼叫者需要管理聊天记录。

我们可以通过将输入和输出消息附加到列表中来实现这一点："chat_history"

In [12]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "什么叫任务分解?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "常见的方法是什么?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

常见的任务分解方法包括使用思维链（Chain of Thought）和思维树（Tree of Thought）。此外，还有LLM+P方法，它结合了大语言模型（LLM）和经典的规划器，通过PDDL（Planning Domain Definition Language）进行问题描述和规划。这些技术帮助模型将复杂任务拆分成可执行的小步骤。


#### 聊天记录的状态管理

在这里，我们已经介绍了如何添加应用程序逻辑以合并历史输出，但我们仍在手动更新聊天历史记录并将其插入到每个输入中。

在真正的问答应用程序中，我们需要某种方式来持久化聊天记录，以及某种自动插入和更新它的方法。

为此，我们可以使用：

`BaseChatMessageHistory`：存储聊天历史记录。

`RunnableWithMessageHistory`：LCEL 链的包装器，用于处理将聊天历史记录注入输入并在每次调用后更新它。`BaseChatMessageHistory`

有关如何将这些类一起使用以创建有状态会话链的详细演练，请转到如何添加消息历史记录（内存）LCEL 页面。

下面，我们实现第二个选项的简单示例，其中聊天记录存储在一个简单的字典中。

LangChain管理与Redis和其他技术的内存集成，以提供更强大的持久性。

为您管理聊天记录的实例。它们接受带有键（默认情况下）的配置，该键指定要提取的对话历史记录并将其附加到输入之前，并将输出附加到相同的对话历史记录。

下面是一个示例：`RunnableWithMessageHistory"session_id" `

In [13]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [14]:
conversational_rag_chain.invoke(
    {"input": "什么是自我反省?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'自我反省是指模型具有检查和评估自身行为及决策过程的能力，特别是在遇到失败时，能识别效率低下和虚构问题，并据此调整策略。在Reflxion框架中，自我反省是通过存储失败轨迹和理想反思的二元组来实现的，这些记忆帮助模型学习如何改进未来的执行序列。'

In [15]:
conversational_rag_chain.invoke(
    {"input": "所以是怎么调整策略呢?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'在Reflxion中，模型通过计算启发式函数$h$来判断动作$a_i$后的效果。如果发现效率低下或问题，模型会选择性地重置环境并开始新试验。通过学习失败的轨迹和理想的反思，模型更新其记忆并在后续决策中避免之前的问题，从而调整策略以提高性能。这种动态记忆和自我反省能力使模型能够适应和优化其行为。'

In [16]:
# 可以在字典中检查对话历史记录：store

for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: 什么是自我反省?

AI: 自我反省是指模型具有检查和评估自身行为及决策过程的能力，特别是在遇到失败时，能识别效率低下和虚构问题，并据此调整策略。在Reflxion框架中，自我反省是通过存储失败轨迹和理想反思的二元组来实现的，这些记忆帮助模型学习如何改进未来的执行序列。

User: 所以是怎么调整策略呢?

AI: 在Reflxion中，模型通过计算启发式函数$h$来判断动作$a_i$后的效果。如果发现效率低下或问题，模型会选择性地重置环境并开始新试验。通过学习失败的轨迹和理想的反思，模型更新其记忆并在后续决策中避免之前的问题，从而调整策略以提高性能。这种动态记忆和自我反省能力使模型能够适应和优化其行为。




![Alternative text](https://python.langchain.com/v0.2/assets/images/conversational_retrieval_chain-5c7a96abe29e582bc575a0a0d63f86b0.png "optional title")

为方便起见，我们将所有必要的步骤绑定在一个代码单元中： (这次我们大模型用智谱吧)

In [55]:
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


os.environ["OPENAI_API_KEY"] = getpass.getpass() # 还是用阿里的啊
os.environ["DASHSCOPE_API_KEY"] = getpass.getpass() # 向量模型也是用阿里

llm = ChatOpenAI(model="qwen-plus", base_url="https://dashscope.aliyuncs.com/compatible-mode/v1", temperature=0)  # 这次，我们还是用阿里


### Construct retriever ###
loader = WebBaseLoader(
    web_paths=("https://tttturtle-russ.github.io/2024/03/04/%E5%9F%BA%E4%BA%8ELLM%E9%A9%B1%E5%8A%A8%E7%9A%84%E8%87%AA%E5%8A%A8%E4%BB%A3%E7%90%86-1-Planning/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=DashScopeEmbeddings())
retriever = vectorstore.as_retriever()


### Contextualize question ###
contextualize_q_system_prompt = (
    " 给定聊天记录和最新的用户问题"
    " 这可能引用了聊天记录中的上下文，"
    " 提出一个可以理解的独立问题 "
    " 没有聊天记录。不要回答问题. "
    " 如果需要，只需重新配制它，否则就按原样返回。"
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [47]:
conversational_rag_chain.invoke(
    {"input": "什么是任务分解?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'任务分解是将一个复杂的大任务分解成多个小任务的过程，使得这些小任务更加容易管理和执行。这可以提高工作效率，确保任务按时完成，并且更容易达成目标。'

In [48]:
conversational_rag_chain.invoke(
    {"input": "常用的方法是什么?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'常用的方法包括思维链（Chain of Thought，CoT）和思维树（Tree of Thought）。思维链是通过引导模型“一步步思考”来将复杂任务分解成小而简单的步骤，而思维树则是在每个步骤后再生成多个想法，形成一个树状结构。此外，还有LLM+P Liu et al. 2023方法，它使用外部经典规划器进行长期规划，通过PDDL（Planning Domain Definition Language）作为接口描述规划问题。'

### Agents 代理

代理利用 LLM 的推理功能在执行过程中做出决策。使用代理可以减轻对检索过程的一些自由裁量权。

尽管它们的行为比链更难预测，但它们在这种情况下提供了一些优势：

- 代理直接生成检索器的输入，而不一定需要我们像上面那样显式地构建上下文化;

- 代理可以执行多个检索步骤来为查询服务，也可以完全不执行检索步骤（例如，响应来自用户的通用问候语）。

#### 检索工具
代理可以访问“工具”并管理其执行。在这种情况下，我们会将检索器转换为由代理使用的 LangChain 工具：

In [56]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "blog_post_retriever",
    "Searches and returns excerpts from the Autonomous Agents blog post.",
)
tools = [tool]

In [57]:
# 工具是 LangChain Runnables，并实现通常的接口：
tool.invoke("任务分解")

'组件一：规划器\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\n\n对于规划器的执行过程主要分两个部分：任务分解（Task Decomposition）和 自我反省（Self-Reflection）。\n对于任务分解文中提到了两个概念：思维链（Chain of Thought，CoT）和思维树（Tree of Thought）。思维链是由 Wei et al. 2022 提出的，已经成为一个增强模型在复杂任务上的标准技术。模型被引导“一步步思考”来利用更多测试时间将复杂任务分解成小而简单的步骤。而思维树Yao et al. 2023，是对于CoT的扩展。其原理是在CoT将复杂问题分解成每个step后，对每个step再生成多个想法，从而形成一个树状结构。对树结构的搜索方法基于给定的prompt或多数票（majority vote）决定为BFS或是DFS。\n其中还提到了一种方法：LLM+P Liu et al. 2023 。主要是使用了一个外部经典规划器来进行长期规划。这种方法使用 PDDL （Planning Domain Definition Language）作为中间接口进行规划问题的描述，主要步骤如下所示：\n\nLLM将问题转化为 “Problem PDDL”\n然后请求外部的经典规划器基于一个已存在的“Domain PDDL”生成一个PDDL规划。\n将 2 中生成的 PDDL 规划翻译成自然语言\n\n组件一：规划器\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\n\n对于规划器的执行过程主要分两个部分：任务分解（Task Decomposition）和 自我反省（Self-Reflection）。\n对于任务分解文中提到了两个概念：思维链（Chain of Thought，CoT）和思维树（Tree of Thought）。思维链是由 Wei et al. 2022 提出的，已经成为一个增强模型在复杂任务上的标准技术。模型被引导“一步步思考

#### Agent 构造函数

现在我们已经定义了工具和 LLM，我们可以创建代理了。

我们将使用 LangGraph 来构建代理。 

目前，我们正在使用一个高级接口来构建代理，但 LangGraph 的好处是，这个高级接口由一个低级、高度可控的 API 支持，以防你想修改代理逻辑。

In [58]:
from langgraph.prebuilt import chat_agent_executor

agent_executor = chat_agent_executor.create_tool_calling_executor(llm, tools)

In [59]:
# 我们现在可以尝试一下了。请注意，到目前为止，它不是有状态的（我们仍然需要添加内存）
query = "什么是任务分解?"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
):
    print(s)
    print("----")


{'agent': {'messages': [AIMessage(content='任务分解（Task Decomposition）是一种将复杂任务拆分为一系列更小、更可管理的子任务的方法。在项目管理、工作流程设计、多智能体系统和人工智能等领域，任务分解是一个重要的概念。通过分解大任务，我们可以更好地理解任务的组成部分，分配工作，设定优先级，以及跟踪各个部分的进度和完成情况。\n\n在不同的上下文中，任务分解可能有不同的形式：\n\n1. **项目管理**：在项目管理中，任务分解通常涉及到工作分解结构（Work Breakdown Structure, WBS），其中项目被逐层分解成更小的组件，直至每个元素都足够详细，可以被单独估算、计划和执行。\n\n2. **多智能体系统**：在人工智能和多智能体系统中，任务分解是指将一个复杂的团队目标转化为每个参与智能体的个体任务，这样它们可以通过协作来共同实现整体目标。\n\n3. **软件工程**：在软件开发过程中，大型功能需求可能会被分解为多个小的编程任务，以便由不同的开发者独立完成。\n\n4. **工作流程**：在日常工作中，复杂的任务可以分解为一系列步骤，便于分配给团队成员，提高效率并确保责任明确。\n\n任务分解有助于提高组织效率，确保所有相关方面都考虑到了，并且可以有效地协调资源和时间。它还可以帮助识别潜在的问题点，提前规划解决方案，并提供更好的沟通和控制机制。', response_metadata={'token_usage': {'completion_tokens': 294, 'prompt_tokens': 195, 'total_tokens': 489}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e770d3a2-0ed7-4999-833c-57e0afb29322-0')]}}
----


LangGraph 内置了持久性，因此我们不需要使用 ChatMessageHistory！

相反，我们可以直接将检查点传递给我们的 LangGraph 代理

In [60]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

agent_executor = chat_agent_executor.create_tool_calling_executor(
    llm, tools, checkpointer=memory
)

这就是我们构建对话式 RAG 代理所需的全部内容。

让我们观察它的行为。请注意，如果我们输入不需要检索步骤的查询，则代理不会执行：

In [61]:
config = {"configurable": {"thread_id": "abc123"}}

for s in agent_executor.stream(
    {"messages": [HumanMessage(content="你好，我是王小明")]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='你好，王小明！有什么问题或者需要帮助的吗？', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 197, 'total_tokens': 211}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9d92fc64-767d-4a60-94fc-58dbcb8be5f0-0')]}}
----


In [62]:
# 此外，如果我们输入的查询确实需要检索步骤，则代理会生成工具的输入：

query = "什么是任务分解?"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='任务分解（Task Decomposition）是一种项目管理或团队协作中的方法，它涉及将一个大任务或复杂项目分成更小、更可管理的子任务。这种方法有助于简化任务，使其更容易理解和执行，同时也便于分配给不同的团队成员或专业领域。任务分解通常在以下几个方面具有重要意义：\n\n1. **明确责任**：通过分解任务，每个人对他们的工作职责有清晰的认识，知道自己的部分在整个项目中的位置和重要性。\n\n2. **提高效率**：小任务可以并行处理，从而加快整个项目的完成速度。多个团队成员可以同时处理不同的子任务，而不是等待一个人完成全部工作。\n\n3. **管理复杂性**：大型任务可能包含许多相互关联的步骤，分解任务可以帮助识别这些步骤之间的依赖关系，并确保所有必要的活动都得到考虑。\n\n4. **跟踪进度**：每个子任务的完成情况可以单独跟踪，这使得监控项目进度和调整计划变得更加容易。\n\n5. **沟通与协调**：任务分解也有助于团队成员之间的沟通，确保每个人都明白自己的角色以及如何与其他人的工作协同。\n\n在实际操作中，任务分解通常使用工作分解结构（Work Breakdown Structure, WBS）来实现，这是一个图形化工具，用于显示任务及其子任务的关系。WBS可以通过层次结构来组织，其中最高级别的元素代表整个项目，而较低级别的元素表示逐步细化的子任务。', response_metadata={'token_usage': {'completion_tokens': 293, 'prompt_tokens': 225, 'total_tokens': 518}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1cac49f0-3d78-4a1d-8b5d-e29ad2394f8e-0')]}}
----


上面，代理没有将我们的查询逐字插入到工具中，而是去除了不必要的单词，如“what”和“is”。

同样的原则允许代理在必要时使用对话的上下文：

In [63]:
query = "根据博客文章，常见的方法是什么？重新搜索。"

try:
    for s in agent_executor.stream({"messages": [HumanMessage(content=query)]}, config=config):
        print(s)
        print("----")
except Exception as e:
    print(f"An error occurred: {e}")


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '', 'function': {'arguments': '{"query": "任务分解 常见方法"}', 'name': 'blog_post_retriever'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 509, 'total_tokens': 535}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-92c980d0-10e3-4129-84c1-6ffdb39d1455-0', tool_calls=[{'name': 'blog_post_retriever', 'args': {'query': '任务分解 常见方法'}, 'id': ''}])]}}
----
{'action': {'messages': [ToolMessage(content='组件一：规划器\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\n\n对于规划器的执行过程主要分两个部分：任务分解（Task Decomposition）和 自我反省（Self-Reflection）。\n对于任务分解文中提到了两个概念：思维链（Chain of Thought，CoT）和思维树（Tree of Thought）。思维链是由 Wei et al. 2022 提出的，已经成为一个增强模型在复杂任务上的标准技术。模型被引导“一步步思考”来利用更多测试时间将复杂任务分解成小而简单的步骤。而思维树Yao et al. 2023，是对于CoT的扩展。其原理是在CoT将复杂问题分解成每个step后，对每个step再生成多个想法，从而形成

请注意，代理能够推断出我们查询中的“it”指的是“任务分解”，并因此生成了一个合理的搜索查询——在本例中为“任务分解的常见方法”。

### 将其绑在一起

In [ ]:
# 为方便起见，我们将所有必要的步骤绑定在一个代码单元中：

import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


os.environ["OPENAI_API_KEY"] = getpass.getpass() # 还是用阿里的啊
os.environ["DASHSCOPE_API_KEY"] = getpass.getpass() # 向量模型也是用阿里

llm = ChatOpenAI(model="qwen-plus", base_url="https://dashscope.aliyuncs.com/compatible-mode/v1", temperature=0)  # 这次，我们还是用阿里


### Construct retriever ###
loader = WebBaseLoader(
    web_paths=("https://tttturtle-russ.github.io/2024/03/04/%E5%9F%BA%E4%BA%8ELLM%E9%A9%B1%E5%8A%A8%E7%9A%84%E8%87%AA%E5%8A%A8%E4%BB%A3%E7%90%86-1-Planning/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


### Build retriever tool ###
tool = create_retriever_tool(
    retriever,
    "blog_post_retriever",
    "Searches and returns excerpts from the Autonomous Agents blog post.",
)
tools = [tool]


agent_executor = chat_agent_executor.create_tool_calling_executor(
    llm, tools, checkpointer=memory
)

### 后续步骤

我们介绍了构建基本对话式 Q&A 应用程序的步骤：

我们使用链来构建一个可预测的应用程序，为每个用户输入生成搜索查询;

我们使用代理来构建一个应用程序，该应用程序可以“决定”何时以及如何生成搜索查询。

要探索不同类型的检索器和检索策略，请访问操作指南的检索器部分。

有关LangChain的对话记忆抽象的详细演练，请访问如何添加消息历史记录（内存）LCEL页面。

要了解有关代理的更多信息，请前往代理模块。